In [1]:
from utils.util import mkdir,seed_all
from omegaconf import OmegaConf
from cprint import *
from datasets.shape_net import ShapeNet
import torch
from models.base_model import BaseModel
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import os
import numpy as np
import datetime
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

## Setup Expirement Folder

In [2]:
seed_all(111)
config = OmegaConf.load("./configs/global_configs.yaml")
cprint.ok(config)
description = "Testing training script" # Describe Experiment params here
logs_dir = config["logs_dir"]
mkdir(logs_dir)

experiment_dir = f"{logs_dir}/{config['name']}"
mkdir(experiment_dir)
mkdir(f"{experiment_dir}/checkpoints")
mkdir(f"{experiment_dir}/tb")
mkdir(f"{experiment_dir}/visuals")

{'logs_dir': 'logs', 'is_train': True, 'name': 'setup3', 'device': 'cuda:0', 'batch_size': 16, 'n_epochs': 1, 'print_every': 50, 'validate_every': 50}
- logs directory found
- logs/setup3 directory found
- logs/setup3/checkpoints directory found
- logs/setup3/tb directory found
- logs/setup3/visuals directory found


## Load Dataset & Dataloaders
This uses a random split for train/validation/test - might need to look into paper if they have a pre-defined split

In [3]:
dataset = ShapeNet(cat="airplane",is_overfit=False) #Change overfit param here & cat here
print('length: ', len(dataset))
train_ds, valid_ds, test_ds = torch.utils.data.random_split(
    dataset, [2830, 810, 405])

train_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )

validation_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )


length:  4045


## Loading Model

In [4]:
model = BaseModel()
# Declare device
device = torch.device('cpu')
if torch.cuda.is_available() and config['device'].startswith('cuda'):
    device = torch.device(config['device'])
    cprint.ok('Using device:', config['device'])
else:
    cprint.warn('Using CPU')

model.to(device)


Using device: cuda:0


## Training

Checklist:
- Add tensorboard and make sure it logs to the `${experiment_dir}/tb` folder
- Visualize some reconstructions on validation set and make sure it logs to the `${expirement_dir}/visuals` folders

In [5]:
train_loss_running = 0.
best_loss_val = np.inf
model.train()

tb_dir = f"{experiment_dir}/tb"
writer = SummaryWriter(log_dir=tb_dir)

#print(experiment_dir)
#print(os.listdir(f"{experiment_dir}/visuals"))

for epoch in tqdm(range(config['n_epochs'])):
    for batch_idx, batch in tqdm(enumerate(train_dataloader)):
        ShapeNet.move_batch_to_device(batch, device)
        model.set_input(batch)
        model.step()
        train_loss_running += model.get_current_errors()
        iteration = epoch * len(train_dataloader) + batch_idx
        #print(iteration)
        if iteration % config['print_every'] == (config['print_every'] - 1):
            avg_train_loss = train_loss_running / config["print_every"]
            cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] train_loss: {avg_train_loss:.6f}')
            writer.add_scalar("Train Loss", avg_train_loss, iteration)
            # cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] train_loss: {train_loss_running / config["print_every_n"]:.6f}')
            train_loss_running = 0.

        if iteration % config['validate_every'] == (config['validate_every'] - 1 or True):
            model.eval()
            loss_val = 0.
            output = []
            
            for batch_val in validation_dataloader:
                ShapeNet.move_batch_to_device(batch_val, device)
                with torch.no_grad():
                    model.set_input(batch_val)
                    output = model.inference()
                    loss_val += model.get_current_errors()

            if loss_val < best_loss_val:
                model_path = f"{experiment_dir}/checkpoints/model_{epoch:03d}.pt"
                model.save(model_path)
                best_loss_val = loss_val

            # cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] val_loss: {loss_val:.6f} | best_loss_val: {best_loss_val:.6f}')
            cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] val_loss: {loss_val:.6f} | best_loss_val: {best_loss_val:.6f}')
            writer.add_scalar("Validation Loss", loss_val, iteration)

            #mkdir(f"{experiment_dir}/visuals")

#             for i in range(len(output)):
#                 visualization = output[i].detach().cpu().numpy()
#                 visualization_path = f"{experiment_dir}/visuals/reconstruction_{epoch:03d}_{i:05d}.png"
#                 visualization = np.transpose(visualization, (1, 2, 0))
#                 visualization = ((visualization + 1) / 2) * 255  # Convert [-1, 1] to [0, 255]
#                 visualization = visualization.astype(np.uint8)
#                 image = Image.fromarray(visualization)
#                 image.save(visualization_path)
                
            
cprint.ok(f"Visualizations saved to: {experiment_dir}/visuals")

model.update_lr()
writer.close()

2023-07-15 16:40:35.778500: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-15 16:40:35.823817: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-15 16:40:37.828244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

BLUB


[000/00049] train_loss: 0.000000


...

BLUB


[000/00099] train_loss: 0.000000
[000/00099] val_loss: 0.000000 | best_loss_val: 0.000000


BLUB


[000/00149] train_loss: 0.000000
[000/00149] val_loss: 0.000000 | best_loss_val: 0.000000


Visualizations saved to: logs/setup3/visuals
